# Controller #

Import all neccessary packages

In [1]:
import os
import pandas as pd
import numpy as np

from controllers.inference_controllers.sandbox import Sandbox
from controllers.inference_controllers.generator import Generator
from controllers.inference_controllers.optimiser import Optimiser

from toolboxes.plotting_toolbox.domain import Domain
from toolboxes.inference_toolbox.parameter import Parameter
from toolboxes.inference_toolbox.model import Model
from toolboxes.inference_toolbox.likelihood import Likelihood

current_directory = os.getcwd()
if current_directory != '/PhD_project/':
    os.chdir('/PhD_project/')

/build/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Defining simulated data

In [2]:
data_params = {
    'data_type': 'normalised_data',
    'data_select': 'GBR_data',
    'normaliser_select': 'GBR_normaliser',
    'normaliser_params':{
        'experiments_list': [
            'Exp1',
            'Exp2',
            'Exp3',
            'Exp4',
            'Exp5',
            'Exp6',
            'Exp7',
            'Exp8',
            'Exp9',
            'Exp10',
            'Exp11',
            'Exp12',
        ],
        'meta_data_select': 'GBR_data_summary',
        'input_header': 'Concentration'
    },
    'log':True,
    'output_header': 'Concentration',
    'gridding': [100,100,25]
}

prior_plots = [
    {
        'Q': [1e12, 5e13],
        'references':{
            'Q' : [2.82e13, 3.11e13, 2.89e13, 4.83e13],
            'labels': ['250m','200m','750m','1000m'] 
        }
    },
    {
        'sigma': [0.001, 4],
    },
    {
        'I_y': [0.0001, 1.7],
        'I_z': [0.0001, 1.7],
        'references':{
            'I_y' : [0.22, 1.6, 0.11, 0.08, 0.06, 0.04],
            'I_z' : [0.2, 1.2, 0.08, 0.06, 0.03, 0.016],
            'labels': ['A','B','C','D','E','F'] 
        }
    }
]

results_name = 'GBR_data_gridded_100_100_25'

Defining default parameters and prior plots for gamma priors of the simulated data

In [3]:
default_params = {
    'infered_params':pd.Series({
        'model_params':pd.Series({
            'I_y_and_I_z': Parameter(['I_y','I_z'], prior_select='log_norm')
                                    .add_prior_param('mu', [0.4,0.4])
                                    .add_prior_param('cov', [[0.8,0],[0,0.8]]),
            'Q': Parameter('Q', prior_select = 'log_norm', order = 1e13).add_prior_param('mu',3.42).add_prior_param('cov',0.01),
        }),
        'likelihood_params':pd.Series({
            'sigma': Parameter('sigma', prior_select = 'log_norm').add_prior_param('mu', 1.5).add_prior_param('cov',0.7)
        })
    }),
    'model':Model('log_gpm_norm').add_fixed_model_param('H',5),
    'likelihood': Likelihood('gaussian'),
    'sampler': {
        'n_samples': 10000,
        'n_chains': 3,
        'thinning_rate': 1
    }
}

Running a general instance for gamma priors and simulated data

In [4]:
sandbox = Sandbox(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

visualiser = sandbox.run()
visualiser.get_summary()
visualiser.get_traceplot()
visualiser.get_autocorrelations()
visualiser.get_prior_plots(prior_plots)

domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 1000)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,5])
domain.add_domain_param('z_slice', 0)
domain.add_domain_param('y_slice', 0)
domain.add_domain_param('x_slice', 100)

visualiser.visualise_results(domain = domain, plot_type = '2D_slice', name = 'small_scale', title='Log Concentration of Droplets', log_results=False)
visualiser.visualise_results(domain = domain, plot_type = '3D', name = 'small_scale', title='Log Concentration of Droplets', log_results=False)

TypeError: Sampler() takes no arguments

In [ ]:
default_params = {
    'infered_params':pd.Series({
        'model_params':pd.Series({
            'I_y_and_I_z': Parameter('I_y','I_z', prior_select='log_norm')
                                    .add_prior_param('mu', [0.4,0.4])
                                    .add_prior_param('cov', [[0.8,0],[0,0.8]]),
            'Q': Parameter('Q', prior_select = 'log_norm', order = 1e13).add_prior_param('mu',3.41).add_prior_param('cov',0.01),
        }),
        'likelihood_params':pd.Series({
            'sigma': Parameter('sigma', prior_select = 'log_norm').add_prior_param('mu', 1.5).add_prior_param('cov',0.7)
        })
    }),
    'model':Model('log_gpm_norm').add_model_param('H',5),
    'likelihood': Likelihood('gaussian'),
    'sampler': {
        'n_samples': 10000,
        'n_chains': 1,
        'thinning_rate': 1
    }
}

TypeError: Parameter.__init__() got multiple values for argument 'prior_select'

Analysing the sensitivity of the gamma prior simulated data

In [ ]:
# generator = Generator(results_name=results_name, 
#                   data_params=data_params,
#                   default_params=default_params)

# analysis_iterations = {
#     'parameters_1': ['I_y_and_I_z_mu_0','I_y_and_I_z_mu_1','Q_mu','sigma_mu'],
#     'parameters_2': ['I_y_and_I_z_cov_0_0','I_y_and_I_z_cov_1_1','Q_sigma','sigma_sigma'],
#     'values_1':
#     [
#         np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
#         np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
#         # np.array([1e9, 1e10, 1e11, 1e12, 1e13, 1e14, 1e15, 1e16, 1e17, 1e18]),
#         # np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5])     
#     ],

#     'values_2':
#     [
#         np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
#         np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]),
#         # np.array([1e7, 1e8, 1e9, 1e10, 1e11, 1e12, 1e13, 1e14, 1e15, 1e16]),
#         # np.array([1e-4, 1e-3, 1e-2, 1e-1, 1e0, 1e1, 1e2, 1e3, 1e4, 1e5]) 
#     ],
#     'scales_1': ['log','log', 'log', 'log'],
#     'scales_2': ['log','log', 'log', 'log'],
# }

# for i in range(len(analysis_iterations['parameters_1'])):
#     parameter_1 = analysis_iterations['parameters_1'][i]
#     parameter_2 = analysis_iterations['parameters_2'][i]

#     print('Working on varying ' + parameter_1 + ' and ' + parameter_2 + '...')
#     values_1 = analysis_iterations['values_1'][i]
#     values_2 = analysis_iterations['values_2'][i]

#     scale_1 = analysis_iterations['scales_1'][i]
#     scale_2 = analysis_iterations['scales_2'][i]

#     inputs = generator.vary_two_parameters(parameter_1, parameter_2, values_1, values_2, scale_1=scale_1, scale_2=scale_2, plot=True)

Optimising the hyperparameters for gamma prior simulated data

In [ ]:
optimising_parameters = {
                    'I_y_and_I_z_mu_0': [1e-2, 2],
                    'I_y_and_I_z_mu_1': [1e-2, 2],
                    # 'I_y_and_I_z_cov_0_0': [1e-2, 2],
                    # 'I_y_and_I_z_cov_1_1': [1e-2, 2],
                }

optimiser = Optimiser(results_name=results_name, 
                  data_params=data_params,
                  default_params=default_params)

# AIC
study = optimiser.run(n_trials=20, optimiser_name='AIC_3', optimising_parameters=optimising_parameters, index_name='aic')
optimiser.get_plots(study)
domain = Domain('cone_from_source_z_limited', resolution=80)
domain.add_domain_param('r', 1000)
domain.add_domain_param('theta', np.pi/8)
domain.add_domain_param('source', [0,0,5])
domain.add_domain_param('z_slice', 0)
domain.add_domain_param('y_slice', 0)
domain.add_domain_param('x_slice', 100)

optimiser.run_best_params(study, domain, name = 'small_scale', prior_plots=prior_plots)

# # # # RMSE
# # # study = optimiser.run(n_trials=100, optimiser_name='RMSE_1', optimising_parameters=optimising_parameters, index_name='rmse')
# # # optimiser.get_plots(study)
# # # domain = Domain('cone_from_source_z_limited', resolution=80)
# # # domain.add_domain_param('r', 1000)
# # # domain.add_domain_param('theta', np.pi/8)
# # # domain.add_domain_param('source', [0,0,10])
# # # optimiser.run_best_params(study, domain, name = 'small_scale_3D_plots', prior_plots=prior_plots)

# # # # BIC
# # # study = optimiser.run(n_trials=100, optimiser_name='BIC_1', optimising_parameters=optimising_parameters, index_name='bic')
# # # optimiser.get_plots(study)
# # # domain = Domain('cone_from_source_z_limited', resolution=80)
# # # domain.add_domain_param('r', 1000)
# # # domain.add_domain_param('theta', np.pi/8)
# # # domain.add_domain_param('source', [0,0,10])
# # # optimiser.run_best_params(study, domain, name = 'small_scale_3D_plots', prior_plots=prior_plots)

[I 2024-04-29 07:13:39,991] Using an existing study with name 'AIC_3' instead of creating a new one.
sample: 100%|██████████| 15000/15000 [00:03<00:00, 4028.67it/s, 7 steps of size 5.37e-01. acc. prob=0.78]
[I 2024-04-29 07:13:44,210] Trial 7 finished with value: 2509.12060546875 and parameters: {'I_y_and_I_z_mu_0': 0.8504878037039512, 'I_y_and_I_z_mu_1': 1.4385781499389758}. Best is trial 3 with value: 2509.005126953125.
sample: 100%|██████████| 15000/15000 [00:03<00:00, 3922.63it/s, 15 steps of size 3.33e-01. acc. prob=0.90]
[I 2024-04-29 07:13:48,448] Trial 8 finished with value: 2509.081298828125 and parameters: {'I_y_and_I_z_mu_0': 1.0350672174531816, 'I_y_and_I_z_mu_1': 1.1097820644115541}. Best is trial 3 with value: 2509.005126953125.
sample: 100%|██████████| 15000/15000 [00:04<00:00, 3415.77it/s, 30 steps of size 6.00e-02. acc. prob=0.93]
[I 2024-04-29 07:13:53,254] Trial 9 finished with value: 2509.13623046875 and parameters: {'I_y_and_I_z_mu_0': 1.3142729015319192, 'I_y_and_